In [11]:
import mediapy as media
import random
import sys
import torch
from diffusers import DiffusionPipeline

# Define whether to use the refiner or not
use_refiner = False  # Set this to False if you don't want to use the refiner

# Load the base model
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

# Load the refiner if use_refiner is True
if use_refiner:
    refiner = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-refiner-1.0",
        text_encoder_2=pipe.text_encoder_2,
        vae=pipe.vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
    )
    refiner = refiner.to("cuda")
    pipe.enable_model_cpu_offload()  # Offload the base model to CPU when using the refiner
else:
    pipe = pipe.to("cuda")

# Function to generate an image from a user-defined prompt
def generate_image(prompt):
    # Generate a random seed
    seed = random.randint(0, sys.maxsize)

    # Generate images
    images = pipe(
        prompt=prompt,
        output_type="latent" if use_refiner else "pil",  # Use "latent" for refiner, else "pil"
        generator=torch.Generator("cuda").manual_seed(seed),
    ).images

    # Refine the images if using the refiner
    if use_refiner:
        images = refiner(
            prompt=prompt,
            image=images,
        ).images

    # Display and save the generated image
    print(f"Prompt:\t{prompt}\nSeed:\t{seed}")
    media.show_images(images)
    images[0].save("output.jpg")
    print("Image saved as 'output.jpg'")

# Main script
if __name__ == "__main__":
    # Take user input for the prompt
    user_prompt = input("Enter your prompt: ")

    # Generate the image
    generate_image(user_prompt)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Enter your prompt: A boy holds hands of girl under moonlight sitting on bench


  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 60.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 56.12 MiB is free. Process 2261 has 14.68 GiB memory in use. Of the allocated memory 14.16 GiB is allocated by PyTorch, and 406.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)